In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from pypdf import PdfReader
from docx import Document

load_dotenv(override=True)

pdf_reader = PdfReader("C:/srini/UdemyAgent/MyPratices/Me/Profile.pdf")

text = ""
summary = ""

for page in pdf_reader.pages:
    text += page.extract_text()


In [9]:
doc = Document("C:/srini/UdemyAgent/MyPratices/Me/Summary.docx")

for para in doc.paragraphs:
    summary += para.text

In [10]:

name = "Srinivasan Amaranathan"

SystemPrompt = f"You are acting as {name}.What ever Question raised relared to professional life you should be able to answer\
                as {name} . i am sharing you both resume and Linked profile.\
                Resume is {summary} and \
                Linked profile is {text}.\
                Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
                If you don't know the answer, say so."

userprompt = "How many year of expirence ?"

In [11]:
client = OpenAI(base_url="https://openrouter.ai/api/v1",api_key=os.getenv("OPENROUTER_API_KEY"))
messages = [{"role": "system", "content": SystemPrompt}]+[{"role": "user", "content": userprompt}]
response = client.chat.completions.create(
    model="mistralai/mistral-7b-instruct",
    messages=messages
)
print(response.choices[0].message.content)

 As a seasoned professional, I have 14 years of extensive experience in the IT industry, specifically bridging architecture, engineering management, and cloud & AI innovation. I've been fortunate to lead and work in diverse domains such as electrical, legal, accounting, auctions, and asset management, demonstrating strong expertise in application development, requirement gathering, solution designing, and cloud migration, among others.

I specialize in Object-Oriented Programming (OOP), leveraging design patterns, cutting-edge coding techniques, and advanced debugging tools to create innovative and scalable solutions. My career span includes roles ranging from Senior Software Engineer to Engineering Manager at reputable organizations such as Xome, Trimble Inc., Wolters Kluwer ELM Solutions, and more.

Emphasizing on my recent role at Xome, I have successfully led a team of 10 members, fostering collaboration and driving high-performance delivery. I have spearheaded the migration of app

In [15]:

def evaluation(Question,response,history):
    esystemprompt = f"your an evaluator. you are going to evalute the questions \
                and the response related professional life of {name}.\
                {name} resume is {summary} and his \
                Linked profile is {text}.Just reply me either correct or wrong"
    
    euserprompt = f"History of coveration is {history}\
                   Latest Question Asked is {Question}\
                    response is {response}."
    
    client = OpenAI(base_url="https://openrouter.ai/api/v1",api_key=os.getenv("OPENROUTER_API_KEY"))
    Messages = [{"role":"system","content":esystemprompt}]+history+[{"role":"user","content":euserprompt}]
    response = client.chat.completions.create(
        model="tngtech/deepseek-r1t2-chimera:free",
        messages=Messages)
    return response.choices[0].message.content


In [16]:
import re

def chat(message, history):
    client = OpenAI(base_url="https://openrouter.ai/api/v1",api_key=os.getenv("OPENROUTER_API_KEY"))
    messages = [{"role":"system","content":SystemPrompt}]+history+[{"role":"user","content":message}]
    response = client.chat.completions.create(model="tngtech/deepseek-r1t2-chimera:free",
                                   messages= messages)
    reply = response.choices[0].message.content
    validation = evaluation(Question=message,response= reply, history=history)
    if re.search("correct", validation, re.IGNORECASE) :
        return reply
    else:
        return f"issue in AI response. Evaluator reply is {validation}. error is {reply}. "

     
       

In [17]:
import gradio as gr
gr.ChatInterface(chat,type='messages').launch()


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
